## Export natureserves to geojson

* testing [umap](https://umap.openstreetmap.fr/en/map/boende-vindskydd-nordsydlinjen-och-stockholm-archi_1257552?share#9/59.3052/18.5161)

see if we can add Naturereserves


In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?_image ?coord ?www ?article ?OSMr ?OSMw ?OSM WHERE {
  ?item wdt:P6104 wd:Q134294510; wdt:P625 ?coord.
  ?item wdt:P31 wd:Q179049.  
  ?item wdt:P856 ?www.
  OPTIONAL { ?item wdt:P402 ?OSMr. }
  OPTIONAL { ?item wdt:P11693 ?OSMw. }
  
  BIND( URI(CONCAT("https://www.openstreetmap.org/way/",?OSMw)) AS ?OSM)
  BIND( URI(CONCAT("https://www.openstreetmap.org/relation/",?OSMr)) AS ?OSM)

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "sv, en" . 
  }
    OPTIONAL { 
   ?article schema:about ?item ;
             schema:inLanguage "sv" ;
             schema:name ?name ;
             schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
  }
OPTIONAL { ?item wdt:P18 ?_image. }
} order by ?objectLabel
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1422492'}, 'article': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Fj%C3%A4rdl%C3%A5ngs_naturreservat'}, 'www': {'type': 'uri', 'value': 'https://www.lansstyrelsen.se/stockholm/besoksmal/naturreservat/fjardlang.html'}, 'coord': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(18.573401117 59.018765892)'}, 'OSMr': {'type': 'literal', 'value': '1463788'}, 'OSM': {'type': 'uri', 'value': 'https://www.openstreetmap.org/relation/1463788'}, '_image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Fjardlang%20P9230090%20%2852103910%29.jpg'}, 'itemLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Fjärdlångs naturreservat'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2301512'}, 'article': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Ut%C3%B6_naturreservat'}, 'www': {'type': 'uri', 'value': 'https://

In [5]:
import sys
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON

# --- Step 1: SPARQL Query ---

endpoint_url = "https://query.wikidata.org/sparql"
query = """
SELECT ?item ?itemLabel ?OSMr ?OSMw WHERE {
  ?item wdt:P6104 wd:Q134294510;  # Stockholm Archipelago Trail
        wdt:P625 ?coord;
        wdt:P31 wd:Q179049.       # nature reserve
  OPTIONAL { ?item wdt:P402 ?OSMr. }    # OSM relation
  OPTIONAL { ?item wdt:P11693 ?OSMw. }  # OSM way
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
"""

def get_results(endpoint_url, query):
    user_agent = "YourCustomAppName/1.0 (contact@example.com)"
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# --- Step 2: Extract OSM IDs ---
osm_ids = []
labels = {}

for r in results["results"]["bindings"]:
    label = r.get("itemLabel", {}).get("value")
    if "OSMr" in r:
        rel_id = r["OSMr"]["value"]
        osm_ids.append(f"relation({rel_id})")
        labels[f"relation/{rel_id}"] = label
    elif "OSMw" in r:
        way_id = r["OSMw"]["value"]
        osm_ids.append(f"way({way_id})")
        labels[f"way/{way_id}"] = label

if not osm_ids:
    print("No OSM IDs found.")
    sys.exit(1)

# --- Step 3: Query Overpass API ---
overpass_query = f"""
[out:json][timeout:25];
(
  {';'.join(osm_ids)};
);
out body;
>;
out skel qt;
"""

response = requests.get("https://overpass-api.de/api/interpreter", params={'data': overpass_query})
overpass_data = response.json()

# --- Step 4: Convert OSM JSON → GeoJSON ---

# Option A: use osmtogeojson (requires Node.js)
# Option B: manual conversion (simplified below)

# Here, we just dump the Overpass output — you can use https://tyrasd.github.io/osmtogeojson/ to convert it
with open("raw_overpass_output.json", "w", encoding="utf-8") as f:
    json.dump(overpass_data, f, indent=2)

print("✅ Overpass JSON saved. You can now convert to GeoJSON using osmtogeojson.")

# Optional: use https://overpass-turbo.eu/ → paste query → Export GeoJSON


✅ Overpass JSON saved. You can now convert to GeoJSON using osmtogeojson.


In [4]:
result

In [8]:
import sys
import requests
import json
from collections import defaultdict
from SPARQLWrapper import SPARQLWrapper, JSON
from geojson import Feature, FeatureCollection, Point, Polygon, LineString

# --- Step 1: Fetch Wikidata items with OSM relation/way IDs ---

def get_results(endpoint_url, query):
    user_agent = "GeoJSONGenerator/1.0 (your@email.com)"
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

endpoint_url = "https://query.wikidata.org/sparql"
query = """
SELECT ?item ?itemLabel ?OSMr ?OSMw ?wwwN WHERE {
  ?item wdt:P6104 wd:Q134294510;
        wdt:P625 ?coord;
        wdt:P31 wd:Q179049.
  OPTIONAL { ?item wdt:P856 ?wwwN . }
  OPTIONAL { ?item wdt:P402 ?OSMr. }
  OPTIONAL { ?item wdt:P11693 ?OSMw. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
"""

results = get_results(endpoint_url, query)

osm_ids = []
labels = {}
wd_qids = {}

for r in results["results"]["bindings"]:
    label = r.get("itemLabel", {}).get("value", "Unnamed")
    qid = r["item"]["value"].split("/")[-1]

    if "OSMr" in r:
        rel_id = r["OSMr"]["value"]
        osm_ids.append(f"relation({rel_id})")
        labels[f"relation/{rel_id}"] = label
        wd_qids[f"relation/{rel_id}"] = qid
    elif "OSMw" in r:
        way_id = r["OSMw"]["value"]
        osm_ids.append(f"way({way_id})")
        labels[f"way/{way_id}"] = label
        wd_qids[f"way/{way_id}"] = qid

if not osm_ids:
    print("No OSM IDs found.")
    sys.exit(1)

# --- Step 2: Query Overpass API ---

overpass_query = f"""
[out:json][timeout:25];
(
  {';'.join(osm_ids)};
);
out body;
>;
out skel qt;
"""

response = requests.get("https://overpass-api.de/api/interpreter", params={'data': overpass_query})
osm_data = response.json()

# --- Step 3: Convert to GeoJSON ---

# Build node dictionary
nodes = {}
for el in osm_data['elements']:
    if el['type'] == 'node':
        nodes[el['id']] = [el['lon'], el['lat']]

features = []

# Build features from ways and relations
for el in osm_data['elements']:
    if el['type'] == 'way':
        coords = []
        for node_id in el['nodes']:
            if node_id in nodes:
                coords.append(nodes[node_id])
        geometry = None
        if coords and coords[0] == coords[-1]:  # closed ring
            geometry = Polygon([coords])
        else:
            geometry = LineString(coords)
        ref = f"way/{el['id']}"
        features.append(Feature(
            geometry=geometry,
            properties={
                "name": labels.get(ref, "Unnamed"),
                "wikidata": wd_qids.get(ref, None),
                "osm_id": ref
            }
        ))

    elif el['type'] == 'relation' and 'members' in el:
        # Simplified support for multipolygon relations
        outer_rings = []
        for member in el['members']:
            if member['type'] == 'way' and member['role'] == 'outer':
                way = next((w for w in osm_data['elements'] if w['type'] == 'way' and w['id'] == member['ref']), None)
                if way:
                    coords = [nodes[nid] for nid in way['nodes'] if nid in nodes]
                    if coords[0] == coords[-1]:  # closed
                        outer_rings.append(coords)
        if outer_rings:
            ref = f"relation/{el['id']}"
            features.append(Feature(
                geometry=Polygon(outer_rings),
                properties={
                    "name": labels.get(ref, "Unnamed"),
                    "wikidata": wd_qids.get(ref, None),
                    "osm_id": ref
                }
            ))

# --- Step 4: Write GeoJSON file ---

geojson_data = FeatureCollection(features)

with open("nature_reserves.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, indent=2, ensure_ascii=False)

print(f"✅ Saved 3 GeoJSON with {len(features)} features to 'nature_reserves.geojson'")


✅ Saved 3 GeoJSON with 85 features to 'nature_reserves.geojson'


In [ ]:
Also 